### TODO
- tc model for two variables with no correlated erros - use state space inference library
- estimate on simulated data
- estimate on real data
- add var error correlation
- add trend correlation
- extedn to more variables

In [66]:
import Pkg
Pkg.activate("../../../FinancialPlanner")

  Activating project at `c:\Users\matsz\programowanie\Optymalizacja_portfela\FinancialPlanner`


In [67]:
using Revise
using LinearAlgebra
using Distributions
using Plots
using StateSpaceInference

includet("TCVAR_model.jl")

In [74]:
params_true = NamedTuple{(:στπ, :στr, :Φ11, :Φ12, :Φ21, :Φ22, :σcπ, :σcr)}((.02, .01, .6, .3, .4, .2, .015, .01))
   

TCVAR(params_true, [.02, .01], diagm([.01, .005]))



LinearGaussianStateSpaceModel{Float64}([0.02, 0.01], [0.01 0.0; 0.0 0.005], [1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 0.6 0.4; 0.0 0.0 0.4 0.2], [0.0, 0.0], [0.02 0.0 0.0 0.0; 0.0 0.01 0.0 0.0; 0.0 0.0 0.015 0.0; 0.0 0.0 0.0 0.01], [1.0 0.0 1.0 0.0; 1.0 1.0 0.0 1.0], [0.0 0.0; 0.0 0.0])

In [73]:
B = [1. 0 1. 0
         1. 1. 0 1.
         ]

2×4 Matrix{Float64}:
 1.0  0.0  1.0  0.0
 1.0  1.0  0.0  1.0